[![AWS Data Wrangler](_static/logo.png "AWS Data Wrangler")](https://github.com/awslabs/aws-data-wrangler)

# 19 - Amazon Athena Cache

[Wrangler](https://github.com/awslabs/aws-data-wrangler) has a cache strategy that is disabled by default and can be enabled passing `max_cache_seconds` biggier than 0. This cache strategy for Amazon Athena can help you to **decrease query times and costs**.

When calling `read_sql_query`, instead of just running the query, we now can verify if the query has been run before. If so, and this last run was within `max_cache_seconds` (a new parameter to `read_sql_query`), we return the same results as last time if they are still available in S3. We have seen this increase performance more than 100x, but the potential is pretty much infinite.

The detailed approach is:
- When `read_sql_query` is called with `max_cache_seconds > 0` (it defaults to 0), we check for the last queries run by the same workgroup (the most we can get without pagination).
- By default it will check the last 50 queries, but you can customize it throught the `max_cache_query_inspections` argument.
- We then sort those queries based on CompletionDateTime, descending
- For each of those queries, we check if their CompletionDateTime is still within the `max_cache_seconds` window. If so, we check if the query string is the same as now (with some smart heuristics to guarantee coverage over both `ctas_approach`es). If they are the same, we check if the last one's results are still on S3, and then return them instead of re-running the query.
- During the whole cache resolution phase, if there is anything wrong, the logic falls back to the usual `read_sql_query` path.

*P.S. The `cache scope is bounded for the current workgroup`, so you will be able to reuse queries results from others colleagues running in the same environment.*

In [1]:
import awswrangler as wr

## Enter your bucket name:

In [2]:
import getpass
bucket = getpass.getpass()
path = f"s3://{bucket}/data/"

 ···········································


## Checking/Creating Glue Catalog Databases

In [3]:
if "awswrangler_test" not in wr.catalog.databases().values:
    wr.catalog.create_database("awswrangler_test")

### Creating a Parquet Table from the NOAA's CSV files

[Reference](https://registry.opendata.aws/noaa-ghcn/)

In [4]:
cols = ["id", "dt", "element", "value", "m_flag", "q_flag", "s_flag", "obs_time"]

df = wr.s3.read_csv(
    path="s3://noaa-ghcn-pds/csv/189",
    names=cols,
    parse_dates=["dt", "obs_time"])  # Read 10 files from the 1890 decade (~1GB)

df

id         dt element  value m_flag q_flag s_flag obs_time
0         AGE00135039 1890-01-01    TMAX    160    NaN    NaN      E      NaN
1         AGE00135039 1890-01-01    TMIN     30    NaN    NaN      E      NaN
2         AGE00135039 1890-01-01    PRCP     45    NaN    NaN      E      NaN
3         AGE00147705 1890-01-01    TMAX    140    NaN    NaN      E      NaN
4         AGE00147705 1890-01-01    TMIN     74    NaN    NaN      E      NaN
...               ...        ...     ...    ...    ...    ...    ...      ...
29240014  UZM00038457 1899-12-31    PRCP     16    NaN    NaN      r      NaN
29240015  UZM00038457 1899-12-31    TAVG    -73    NaN    NaN      r      NaN
29240016  UZM00038618 1899-12-31    TMIN    -76    NaN    NaN      r      NaN
29240017  UZM00038618 1899-12-31    PRCP      0    NaN    NaN      r      NaN
29240018  UZM00038618 1899-12-31    TAVG    -60    NaN    NaN      r      NaN

[29240019 rows x 8 columns]

In [5]:
wr.s3.to_parquet(
    df=df,
    path=path,
    dataset=True,
    mode="overwrite",
    database="awswrangler_test",
    table="noaa"
);

In [6]:
wr.catalog.table(database="awswrangler_test", table="noaa")

Column Name       Type  Partition Comment
0          id     string      False        
1          dt  timestamp      False        
2     element     string      False        
3       value     bigint      False        
4      m_flag     string      False        
5      q_flag     string      False        
6      s_flag     string      False        
7    obs_time     string      False

## The test query

The more computational resources the query needs, the more the cache will help you. That's why we're doing it using this long running query.

In [7]:
query = """
SELECT
    n1.element,
    count(1) as cnt
FROM
    noaa n1
JOIN
    noaa n2
ON
    n1.id = n2.id
GROUP BY
    n1.element
"""

## First execution...

In [8]:
%%time

wr.athena.read_sql_query(query, database="awswrangler_test")

CPU times: user 5.31 s, sys: 232 ms, total: 5.54 s
Wall time: 6min 42s


element          cnt
0     WDMV     49755046
1     SNWD   5089486328
2     DATN     10817510
3     DAPR    102579666
4     MDTN     10817510
5     WT03     71184687
6     WT09       584412
7     TOBS    146984266
8     DASF      7764526
9     WT04      9648963
10    WT18     92635444
11    WT01     87526136
12    WT16    323354156
13    PRCP  71238907298
14    SNOW  21950890838
15    WT06       307339
16    TAVG   2340863803
17    TMIN  41450979633
18    MDTX     11210687
19    WT07      4486872
20    WT10       137873
21    EVAP       970404
22    WT14      8073701
23    DATX     11210687
24    WT08     33933005
25    WT05      8211491
26    TMAX  39876132467
27    MDPR    114320989
28    WT11     22212890
29    DWPR     69005655
30    MDSF     12004843

## Second execution with **CACHE** (400x faster)

In [9]:
%%time

wr.athena.read_sql_query(query, database="awswrangler_test", max_cache_seconds=900)

CPU times: user 493 ms, sys: 34.9 ms, total: 528 ms
Wall time: 975 ms


element          cnt
0     WDMV     49755046
1     SNWD   5089486328
2     DATN     10817510
3     DAPR    102579666
4     MDTN     10817510
5     WT03     71184687
6     WT09       584412
7     TOBS    146984266
8     DASF      7764526
9     WT04      9648963
10    WT18     92635444
11    WT01     87526136
12    WT16    323354156
13    PRCP  71238907298
14    SNOW  21950890838
15    WT06       307339
16    TAVG   2340863803
17    TMIN  41450979633
18    MDTX     11210687
19    WT07      4486872
20    WT10       137873
21    EVAP       970404
22    WT14      8073701
23    DATX     11210687
24    WT08     33933005
25    WT05      8211491
26    TMAX  39876132467
27    MDPR    114320989
28    WT11     22212890
29    DWPR     69005655
30    MDSF     12004843

## Allowing Wrangler to inspect up to 500 historical queries to find same result to reuse.

In [10]:
%%time

wr.athena.read_sql_query(query, database="awswrangler_test", max_cache_seconds=900, max_cache_query_inspections=500)

CPU times: user 504 ms, sys: 44 ms, total: 548 ms
Wall time: 1.19 s


element          cnt
0     WDMV     49755046
1     SNWD   5089486328
2     DATN     10817510
3     DAPR    102579666
4     MDTN     10817510
5     WT03     71184687
6     WT09       584412
7     TOBS    146984266
8     DASF      7764526
9     WT04      9648963
10    WT18     92635444
11    WT01     87526136
12    WT16    323354156
13    PRCP  71238907298
14    SNOW  21950890838
15    WT06       307339
16    TAVG   2340863803
17    TMIN  41450979633
18    MDTX     11210687
19    WT07      4486872
20    WT10       137873
21    EVAP       970404
22    WT14      8073701
23    DATX     11210687
24    WT08     33933005
25    WT05      8211491
26    TMAX  39876132467
27    MDPR    114320989
28    WT11     22212890
29    DWPR     69005655
30    MDSF     12004843

## Cleaning Up S3

In [11]:
wr.s3.delete_objects(path)

## Delete table

In [12]:
wr.catalog.delete_table_if_exists(database="awswrangler_test", table="noaa")

True

## Delete Database

In [13]:
wr.catalog.delete_database('awswrangler_test')